In [3]:
import altair as alt
import pandas as pd
import eco_style
alt.themes.enable('light')
import json

In [4]:
data = json.load(open('data.json'))['data']

In [50]:
df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['label'], format="%b-%y")
df['Up to 18 weeks'] = df['value'].apply(lambda x: x[0])
df['Between 18-52 weeks'] = df['value'].apply(lambda x: x[1])
df['Over 52 weeks'] = df['value'].apply(lambda x: x[2])

df = df[['date', 'Up to 18 weeks', 'Between 18-52 weeks', 'Over 52 weeks']]
df = df.melt('date', var_name='series', value_name='value')
df['value'] = df['value'].replace(',', '', regex=True).astype(int)

order = {
    'Up to 18 weeks': 0,
    'Between 18-52 weeks': 1,
    'Over 52 weeks': 2
}

df['order'] = df['series'].map(order)

df = df.sort_values('order')

base = alt.Chart(df).encode(
    x=alt.X('date:T', axis=alt.Axis(title=None)),
    y=alt.Y('value:Q', axis=alt.Axis(title=None), title=None),
    color=alt.Color('series:N',
                    legend=alt.Legend(
                        orient='top',
                        title=None),
                    scale=alt.Scale(reverse=True,
                                    range=[
                                        "rgb(73, 191, 161)",

                                        "rgb(111, 181, 219)",
                                        "rgb(230, 189, 81)",
                                    ]


                                    ),
                    ),
    order='order'
)

# put a line at 2020-03-23
rule = alt.Chart(pd.DataFrame([
    {'date': '2020-03-23'}
])).mark_rule(
    strokeDash=[5, 5],
    color='black').encode(
    x='date:T'
)

# put some text on the line - Start of the pandemic
text = alt.Chart(pd.DataFrame([
    {'date': '2020-03-23', 'label': 'Start of the pandemic'}
])).mark_text(dy=-10, dx=-60, color='black').encode(
    x='date:T',
    text='label',
    y=alt.value(20)
)

area = base.mark_area(opacity=1).encode(
    y=alt.Y('value:Q', stack='zero', axis=alt.Axis(title=None)),
)

chart = area + rule + text

chart = chart.properties(
    width=400,
    height=300,
    title={
        "text": ["Waiting time for consultant-led elective care"],
        "dx": 55,
        "anchor": "start",
        "subtitle": ["August 2007 to March 2024", "Source: BMA analysis of NHS data", ""],
        "subtitleColor": "#676A86",
    }
)

chart.save('elective_care_waiting_times.json')
chart.save('elective_care_waiting_times.png', scale_factor=3.0)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

In [13]:
df

,date,series,value
0,2007-08-01,Up to 18 weeks,"2,395,656"
1,2007-09-01,Up to 18 weeks,"2,364,349"
2,2007-10-01,Up to 18 weeks,"2,337,604"
3,2007-11-01,Up to 18 weeks,"2,269,968"
4,2007-12-01,Up to 18 weeks,"2,180,207"
...,...,...,...
595,2023-11-01,Over 52 weeks,"355,412"
596,2023-12-01,Over 52 weeks,"337,450"
597,2024-01-01,Over 52 weeks,"321,394"
598,2024-02-01,Over 52 weeks,"305,050"
